---------------------------------------------------------------------------
SkyModel.py
Calculates a multipoint illumination hillshade raster from text file input.
Based on model developed by Kennelly and Stewart, 2013
Programmed by Linda Beale, Esri Inc
---------------------------------------------------------------------------

In [ ]:
import os
import sys
import arcpy
import csv
from arcpy import env
from arcpy.sa import *

In [ ]:
def skymodel(inDEM, inFile, outRaster):
    """
    skymodel: calculates hillshade raster

    Required arguments:
    inDEM -- Input DEM.
    inFile -- input text file.
    outRaster -- output Raster.
    """
    try:
        # Check out the Spatial Analyst license
        arcpy.CheckOutExtension("Spatial")

        # Create a blank raster fror the area.
        zeroRaster = Times(inDEM, 0)
        count = 0
        timesDEM = Raster(inDEM) * 5

        # Open file and iterate through file line by line.
        with open(inFile, "r") as in_file:
            reader = csv.reader(in_file)
            for line in reader:
                # splits the line at comma.
                azimuth = line[0]
                altitude = line[1]
                weight = float(line[2])

                # Hillshade input with azimuth and altitude from text file
                outHillShade = Hillshade(timesDEM, azimuth, altitude, "SHADOWS", 1)

                # Weight input with weight from text file.
                weightRas = outHillShade * weight

                # Sum weighted grids.
                if count == 0:
                    sumRaster = weightRas + zeroRaster
                else:
                    sumRaster = weightRas + prevRaster

                # Create output raster
                prevRaster = sumRaster

                # Increment to next text file line.
                count += 1
                print(str(count))

        # Save output file.
        prevRaster.save(outRaster)

        # set the symbology
        scriptPath = sys.path[0]
        one_folder_up = os.path.dirname(scriptPath)
        toolLayerPath = os.path.join(one_folder_up, "LayerFiles")
        lyrFile = os.path.join(toolLayerPath, "SkyModel.lyr")
        params = arcpy.GetParameterInfo()
        params[2].symbology = lyrFile

        arcpy.Delete_management(timesDEM)

    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
# End main function
md = r"C:\Demos\Cartography\Cartography.gdb\New_Zealand_DEM"
if __name__ == '__main__':
    skymodel(md, r"C:\Users\tim10393\Downloads\TerrainTools\TerrainTools_v1_1\SkyLuminance\model_clear.txt", r"C:\Demos\Cartography\New Zealand\Surface Products\SkyModel.tif")
